In [15]:
import os
import sys
import csv
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix as cmat
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report as report  
from sklearn.model_selection import *
from sklearn.feature_selection import *
import matplotlib.pyplot as plt
import time
import random
import numpy
from datetime import datetime
import pandas as pd
import requests
import re
import xarray as xr
import pickle

In [3]:
#Papa CTD Data:

url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/mvalera-w@sdsu.edu/20180822T010245-GP02HYPM-WFP02-04-CTDPFL000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
        
datasets = [os.path.join(tds_url, i) for i in x]
datasets

# Chunk it:

ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.chunk({'time': 100})
ds = ds.sortby('time') # data from different deployments can overlap so we want to sort all data by time stamp.
# Dropping not needed coordinates:
ds2 = ds.drop(['time','obs','pressure','lat','lon'], dim=None)
ds2

g_temp = ds2.ctdpf_ckl_seawater_temperature.to_dataframe()
#g_temp = g_temp.rename(columns={'ctdpf_ckl_seawater_temperature': 'temp'})

g_salinity = ds2.practical_salinity.to_dataframe()
#g_salinity = g_salinity.rename(columns={'practical_salinity': 'sal'})

g_pressure = ds2.ctdpf_ckl_seawater_pressure.to_dataframe()

g_conductivity = ds2.ctdpf_ckl_seawater_conductivity.to_dataframe()

g_density = ds2.density.to_dataframe()

# Joining tables to create dataset:
papa_ctd = pd.concat([g_temp, g_salinity, g_conductivity, g_density],axis=1, join_axes=[g_temp.index])

#Add a column for labeling: 

papa_ctd['Label']=1


In [18]:
papa_rows = papa_ctd['Label'].shape

# Decimate to 10% to make it faster:

bins = int(.1*papa_rows[0]) #Saving only 10% because is just too much

papa_file = papa_ctd.groupby(pd.qcut(papa_ctd.index,bins)).mean()


papa_file.to_pickle('papa_file.pkl')

In [24]:
#GS Dataset:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/mvalera-w@sdsu.edu/20180822T185654-GS02HYPM-WFP02-04-CTDPFL000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
        
datasets = [os.path.join(tds_url, i) for i in x]
datasets

# Chunk it:

ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.chunk({'time': 100})
ds = ds.sortby('time') # data from different deployments can overlap so we want to sort all data by time stamp.
# Dropping not needed coordinates:
ds3 = ds.drop(['time','obs','pressure','lat','lon'], dim=None)
ds3

g_temp = ds3.ctdpf_ckl_seawater_temperature.to_dataframe()
#g_temp = g_temp.rename(columns={'ctdpf_ckl_seawater_temperature': 'temp'})

g_salinity = ds3.practical_salinity.to_dataframe()
#g_salinity = g_salinity.rename(columns={'practical_salinity': 'sal'})

g_pressure = ds3.ctdpf_ckl_seawater_pressure.to_dataframe()

g_conductivity = ds3.ctdpf_ckl_seawater_conductivity.to_dataframe()

g_density = ds3.density.to_dataframe()

# Joining tables to create dataset:
gs_ctd = pd.concat([g_temp, g_salinity, g_conductivity, g_density],axis=1, join_axes=[g_temp.index])
gs_ctd

#Add a column for labeling: 

gs_ctd['Label']=0
gs_ctd

,ctdpf_ckl_seawater_temperature,practical_salinity,ctdpf_ckl_seawater_conductivity,density,Label
time,,,,,
0,5.4183,32.848982,3.20664,1027.109042,0
1,5.4178,32.849688,3.20666,1027.109753,0
2,5.4177,32.849667,3.20665,1027.109795,0
3,5.4177,32.849914,3.20667,1027.109805,0
4,5.4179,32.849832,3.20668,1027.109716,0
5,5.4180,32.849502,3.20666,1027.109489,0
6,5.4177,32.849845,3.20666,1027.109335,0
7,5.4182,32.850260,3.20674,1027.109648,0
8,5.4194,32.849211,3.20675,1027.108581,0


In [25]:
gs_rows = gs_ctd['Label'].shape

# Decimate to 10% to make it faster:

bins = int(.1*gs_rows[0]) #Saving only 10% because is just too much

gs_file = gs_ctd.groupby(pd.qcut(gs_ctd.index,bins)).mean()


gs_file.to_pickle('gs_file.pkl')